Create a file with data from closest grid point
===

Import required modules and set file names
----

In [1]:
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np
import os

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

Model and period-specific data
----

In [124]:
# key data
model = "cesmle"

# branch = "historical"
branch = "rcp85"

# mon = 6
# mon = 7
# mon = 8
mon = "JJA"

# suff = None
suff = "adj"

nEns = 10

# ------------------------------------
# derived data
if branch == "historical":
    yr1 = "1996"
    yr2 = "2005"

if branch == "rcp85":
    yr1 = "2071"
    yr2 = "2080"

try:
    mms = "%02d" % mon
except:
    mms = mon

yrs = yr1+"-"+yr2

File templates etc
---

In [125]:
modelMetaFN = "wrf_geog.nc"

modelDataDirTemplate = "/Volumes/sbp1/model/pwrf/gis_%s/%s/wrf/postproc/tas/%%03d" % (model, branch )
if suff is None:
    modelDataTemplate = "tas_wrf_%s_%%03d_%s_%s_d.nc" % ( model, yrs, mms )
    newNCFN = "wrf_%s_mbr_tas_closest_%s_%s.nc" % ( model, yrs, mms )
    ensNCFN = "wrf_%s_ens_tas_closest_%s_%s.nc" % ( model, yrs, mms )
else:
    modelDataTemplate = "tas_wrf_%s_%%03d_%s_%s_d_%s.nc" % ( model, yrs, mms, suff )
    newNCFN = "wrf_%s_mbr_tas_closest_%s_%s_adj.nc" % ( model, yrs, mms )
    ensNCFN = "wrf_%s_ens_tas_closest_%s_%s_adj.nc" % ( model, yrs, mms )

print newNCFN
print ensNCFN

wrf_cesmle_mbr_tas_closest_2071-2080_JJA_adj.nc
wrf_cesmle_ens_tas_closest_2071-2080_JJA_adj.nc


Load/define CESM LE metadata
----

In [126]:
M = Model(modelMetaFN)
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

2d dataset


Load AWS metadata
----

In [127]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points
-----

In [128]:
C = []
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    C.append( closest )

Process tas data
----

In [129]:
for ens in range(nEns):
# for ens in ([0,1]):
# for ens in ([0]):
    enss = ens+1
    modelDataDir = modelDataDirTemplate % enss
    modelDataFN = modelDataTemplate % enss
    print modelDataFN
    D = Model(None, modelDataDir+"/"+modelDataFN)

    tas = np.array( D.loadData( "tas" ) )
    nRec, nLat, nLon = tas.shape
    print nRec, nLat, nLon

    if ens < 1:
        time, timeCF = D.loadData( "time" )
        Tnew = np.empty( (nEns, nRec, nSites), dtype = 'float' )
    print time[0], time[-1]
    
    for awsIX in (range(nSites)):
    # for awsIX in ((0,1)):
    # for awsIX in ([0]):
        """ Basic AWS info """
        awsName = A.getName( awsIX )
        awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))

        closest = C[awsIX][0]
        dx = "%.1f km" % closest[-1]
        print awsName, dx
#         print Tnew[0][0][0]
        Tnew[ens,:,awsIX] = tas[:,closest['ixLat'],closest['ixLon'] ]
#         print Tnew[0][0][0]
    print " "


tas_wrf_cesmle_001_2071-2080_JJA_d_adj.nc
920 219 197
2071-06-01 10:30:00 2080-08-31 10:30:00
Swiss Camp 2.1 km
Crawford Point1 5.6 km
NASA-U 6.7 km
GITS 6.4 km
Humboldt 6.9 km
Summit 6.9 km
Tunu-N 4.2 km
DYE-2 3.8 km
JAR1 7.7 km
Saddle 5.6 km
South Dome 6.1 km
NASA-E 3.1 km
Crawford Point2 10.0 km
NGRIP 6.2 km
NASA-SE 7.2 km
KAR 1.2 km
JAR2 4.4 km
KULU 5.5 km
JAR3 1.0 km
Aurora 3.9 km
Petermann GL 6.5 km
Petermann ELA 4.9 km
NEEM 2.7 km
 
tas_wrf_cesmle_002_2071-2080_JJA_d_adj.nc
920 219 197
2071-06-01 10:30:00 2080-08-31 10:30:00
Swiss Camp 2.1 km
Crawford Point1 5.6 km
NASA-U 6.7 km
GITS 6.4 km
Humboldt 6.9 km
Summit 6.9 km
Tunu-N 4.2 km
DYE-2 3.8 km
JAR1 7.7 km
Saddle 5.6 km
South Dome 6.1 km
NASA-E 3.1 km
Crawford Point2 10.0 km
NGRIP 6.2 km
NASA-SE 7.2 km
KAR 1.2 km
JAR2 4.4 km
KULU 5.5 km
JAR3 1.0 km
Aurora 3.9 km
Petermann GL 6.5 km
Petermann ELA 4.9 km
NEEM 2.7 km
 
tas_wrf_cesmle_003_2071-2080_JJA_d_adj.nc
920 219 197
2071-06-01 10:30:00 2080-08-31 10:30:00
Swiss Camp 2.1 km


Create a netCDF file for the new data
-----

In [130]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

print "Creating "+newNCFN
ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Closest-point temperature data"
ncfile.period = yrs

time_dim = ncfile.createDimension('time', None)
ens_dim = ncfile.createDimension('ensemble', nEns)
stn_dim = ncfile.createDimension('station', nSites)

timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis

timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

# tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
tasVar = ncfile.createVariable('tas', np.float32, ('ensemble','time','station'))
# tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
tasVar.units = "deg C"
tasVar.description = "Surface temperature, closest point"
tasVar.long_name = "Reference height temperature"
# tasVar[:,:] = Tnew
tasVar[:] = Tnew

ncfile.close()

Creating wrf_cesmle_mbr_tas_closest_2071-2080_JJA_adj.nc


Create ensemble average
----

In [131]:
print "Creating "+ensNCFN
cmd = 'ncwa -O -a ensemble '+newNCFN+' '+ensNCFN
rc = os.system( cmd )

Creating wrf_cesmle_ens_tas_closest_2071-2080_JJA_adj.nc
